In [14]:
import argparse
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset, replace_ImageToTensor
from mmdet.models import build_detector
from mmdet.utils import (
    build_ddp,
    build_dp,
    compat_cfg,
    get_device,
    replace_cfg_vals,
    setup_multi_processes,
    update_data_root,
)
import subprocess
import random

In [15]:
CONFIG = "/media/quadro/NVME/Mehrab/Cycle_Centernet_32_quad_long/config.py"
CHECKPOINT = "/media/quadro/NVME/Mehrab/exps/32_quad_long_retry/latest.pth"
OUTPUT_PATH = "/media/quadro/NVME/Mehrab/exps/32_quad_long_retry/test_output.pkl"
EXP_DIR = "/media/quadro/NVME/Mehrab/exps/32_quad_long_retry"

In [16]:
cfg = Config.fromfile(CONFIG)

# replace the ${key} with the value of cfg.key
cfg = replace_cfg_vals(cfg)

# update data root according to MMDET_DATASETS
update_data_root(cfg)

cfg = compat_cfg(cfg)

# set multi-process settings
setup_multi_processes(cfg)

In [17]:
# set cudnn_benchmark
if cfg.get("cudnn_benchmark", False):
    torch.backends.cudnn.benchmark = True

if "pretrained" in cfg.model:
    cfg.model.pretrained = None
elif "init_cfg" in cfg.model.backbone:
    cfg.model.backbone.init_cfg = None

if cfg.model.get("neck"):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get("rfp_backbone"):
                if neck_cfg.rfp_backbone.get("pretrained"):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get("rfp_backbone"):
        if cfg.model.neck.rfp_backbone.get("pretrained"):
            cfg.model.neck.rfp_backbone.pretrained = None

In [18]:
cfg.gpu_ids = [3]

cfg.device = get_device()
# init distributed env first, since logger depends on the dist info.

distributed = False


test_dataloader_default_args = dict(samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)


In [19]:
# in case the test dataset is concatenated
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    if cfg.data.test_dataloader.get("samples_per_gpu", 1) > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    if cfg.data.test_dataloader.get("samples_per_gpu", 1) > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

In [20]:
test_loader_cfg = {**test_dataloader_default_args, **cfg.data.get("test_dataloader", {})}


In [21]:
rank, _ = get_dist_info()

In [22]:
# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

loading annotations into memory...
Done (t=4.84s)
creating index...
index created!


In [23]:
# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get("test_cfg"))
fp16_cfg = cfg.get("fp16", None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, CHECKPOINT, map_location="cpu")

2024-01-13 14:11:50,017 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.0.conv is upgraded to version 2.
2024-01-13 14:11:50,019 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.2.conv is upgraded to version 2.
2024-01-13 14:11:50,021 - root - INFO - ModulatedDeformConvPack neck.deconv_layers.4.conv is upgraded to version 2.


load checkpoint from local path: /media/quadro/NVME/Mehrab/exps/32_quad_long_retry/latest.pth


In [28]:
# old versions did not save class info in checkpoints, this walkaround is
# for backward compatibility
if "CLASSES" in checkpoint.get("meta", {}):
    model.CLASSES = checkpoint["meta"]["CLASSES"]
else:
    model.CLASSES = dataset.CLASSES
    # model.CLASSES = dataset[0].CLASSES



In [34]:
model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids) 
    


In [35]:
outputs = single_gpu_test(model, data_loader, False, EXP_DIR, 0.3)

[                                                  ] 0/3611, elapsed: 0s, ETA:torch.Size([1, 3000, 9]) torch.Size([1, 3000])
before in _get_bboxes_single batch_det_bboxes = torch.Size([1, 3000, 9]), batch_labels = torch.Size([1, 3000])
before in _get_bboxes_single batch_det_bboxes = torch.Size([1, 3000, 9]), batch_labels = torch.Size([1, 3000])
before in _get_bboxes_single batch_det_bboxes = torch.Size([1, 3000, 9]), batch_labels = torch.Size([1, 3000])
before in _get_bboxes_single batch_det_bboxes = torch.Size([1, 3000, 9]), batch_labels = torch.Size([1, 3000])
before in _get_bboxes_single batch_det_bboxes = torch.Size([1, 3000, 9]), batch_labels = torch.Size([1, 3000])


IndexError: boolean index did not match indexed array along dimension 0; dimension is 5400 but corresponding boolean dimension is 3000

In [33]:
rank, _ = get_dist_info()
if rank == 0:
    if True:
        print(f"\nwriting results to {OUTPUT_PATH}")
        mmcv.dump(outputs, OUTPUT_PATH)
    kwargs = {}

    if True:
        eval_kwargs = cfg.get("evaluation", {}).copy()
        # hard-code way to remove EvalHook args
        for key in ["interval", "tmpdir", "start", "gpu_collect", "save_best", "rule", "dynamic_intervals"]:
            eval_kwargs.pop(key, None)
        eval_kwargs.update(dict(metric="bbox", **kwargs))
        metric = dataset.evaluate(outputs, **eval_kwargs)
        print(metric)
        metric_dict = dict(config=cfg, metric=metric)
        if args.work_dir is not None and rank == 0:
            mmcv.dump(metric_dict, json_file)


writing results to /media/quadro/NVME/Mehrab/exps/32_quad_long_retry/test_output.pkl


NameError: name 'outputs' is not defined